# Install MQTT


MQTT is a lightweight publish/ subscribe messaging protocol. It can be used in IoT scenarios where the sensor data is communicated through a broker to a server which can be a database. In the previous ZMQ examples we have seen the actors communicating messages amongst themselves, there were no brokers involved. However, through the second assignment we have also created a broker between the two actors. The broker we had was very simple, it was acting as a pipeline to pass the messages. However in distributed applications, broker plays a very important role in scheduling messages, load balancing, etc. MQTT allows the luxury to have broker services online or on local host.

In [ ]:
!pip install paho-mqtt

#Broker

A number of brokers are available online like "test.mosquitto.org", "iot.eclipse.org" which can be used. However, they may be offline sometimes and may not work. So, it is best to install a broker on the localhost using:

In [ ]:
!apt-get install mosquitto

In [3]:
!service mosquitto start

 * Starting network daemon: mosquitto
   ...done.


# Subscriber


In this example the server subscribes to a specific topic and the message is received from the MQTT broker. The broker receives the message from the client and stores it, the data then can be subscribed by a single/ multiple servers.

Broker is the MQTT broker service. This service can either be online/ on the local host. There is a list of online available brokers which can be used if you do not want to have a local machine to run the broker.

Topic is message for which the server wants to listen for. The server can subscribe to a single/ multiple topics.

Client = mqtt.Client(“Server”) creates a server instance of MQTT type.

Client.on_message is used to call a function every time a message is received. In the above example, the message received will be printed on the terminal. This function can be anything.

Client.connect(broker) connects to the broker.

Client.subscribe(“Topic”) subscribes to the messages under the topic defined.

In [5]:
import random
import time
import os
import paho.mqtt.client as mqtt
broker = "127.0.0.1"
port = 2000
topic = "RDS19" #topic to be used for communication. You can change this as you wish

# On receiving the message print it
def on_message(client, userdata, message):
    print(f"Message received by subscriber on topic {topic}: {message.payload.decode('utf-8')}")

#On connecting to the broker subscribe to the topic
def on_connect(client, userdata, flags, rc):
    # print('Subscriber connected to the broker.')
    client.subscribe(topic)

def subscriber():
    mqtt.Client.connected_flag=False#create flag in class
    client = mqtt.Client("Server")#create new instance
    client.on_message= on_message#attach function to callback
    client.on_connect = on_connect#attach a callback on connect
   #bind call back function
    client.loop_start()#start a loop
    print("Connecting to broker ",broker)
    client.connect(broker)#connect to broker

# Sometimes it takes a while for the mosquitto server to start (especially 
# if you press the run all menu). So wait out a bit before trying to connect.
sleep(5)

subscriber()

Connecting to broker  127.0.0.1


# Publisher

The publisher connects to the MQTT broker and publishes data (on the same topic) which can then be subscribed at a later time by the server.



In [6]:
#on connect check if you are still conected to the broker
def on_publisher_connect(client, userdata, flags, rc):
    if rc==0:
        client.connected_flag=True #set flag
        print("Publisher connected OK")
    else:
        print("Publisher Bad connection Returned code=",rc)

def publisher():
    mqtt.Client.connected_flag=False#create flag in class
    client = mqtt.Client("client1")#create new instance
    client.on_connect=on_publisher_connect  #bind call back function
    client.loop_start()#start the loop
    print("Publisher Connecting to broker ",broker)
    client.connect(broker)#connect to the broker
    while not client.connected_flag: #wait in loop
        print("In wait loop")
        time.sleep(1)
    print("Main Loop")
    while True:
        publish_string = input("Enter string to publish (Enter quit to stop)")
        if publish_string.lower() == 'quit': break
        client.publish(topic, publish_string)#keep publishing in the loop
        print("Message published")
        time.sleep(2)
    client.disconnect()#Disconnect when done

publisher()

Publisher Connecting to broker  127.0.0.1
In wait loop
In wait loopPublisher connected OK

Main Loop
Enter string to publish (Enter quit to stop)abcd
Message published
Message received by subscriber on topic RDS19: abcd
Enter string to publish (Enter quit to stop)quit
